In [63]:
import numpy as np
import matplotlib.pyplot as plt

In [64]:
# ------------------------------------------------------------------------------
# Parameters & Constants for Diffusion 
# ------------------------------------------------------------------------------

from scipy.constants import epsilon_0, elementary_charge as e, Avogadro as NA, k as kB

epsilon_r = 40.0 #relative permittivity
epsilon = epsilon_0 * epsilon_r #absolute permittivity, F/m
i_a = 0 #-8e6           # Applied current density [A/m²]
sigma = 1.0          #conductivity (for the current density to electric field proportionality)

# ------------------------------------------------------------------------------
# Reference values from literature
# ------------------------------------------------------------------------------
c_ref = 1660               # mol/m^3, reference concentration
rho_ref = 1.6e6            # C/m^3, reference charge density
interface_width = 4.0      # nm, width of the phase-field interface (0 < C < 1)
lambda_D = 1.0             # nm, Debye length
T = 1273.0                 # K, temperature

In [65]:
# make a dictionary with the dict () constructor for all species

charges = {
    "vac" : 2.0,
    "elec" : -1.0,
    "yzr" : -1.0,
}

# concentrations -> [mol/m³]

vac = {
    "mu_YSZ" : 0.12,
    "mu_anode" : 0.2,
    "diffusivity" : 1.0e-8,
    "bulk_anode_conc" : 83.0,
    "bulk_YSZ_conc" : 830.0,
    "rate_constant" : abs(i_a / e * NA * charges["vac"]),
}

elec = {
    "mu_YSZ" : 0.0,
    "mu_anode" : 0.0,
    "diffusivity" : 2.0e-4,
    "bulk_YSZ_conc" : 0.0,
    "bulk_anode_conc" : charges["vac"] * vac["bulk_anode_conc"],
    "rate_constant" : abs(i_a / e * NA * charges["elec"]),
}

yzr = {
    "mu_YSZ" : 0.0,
    "mu_anode" : 0.0,
    "diffusivity" : 1.0e-8,
    "bulk_YSZ_conc" : 1660.0,
    "bulk_anode_conc" : 0.0,
    "rate_constant" : 0.0,
}

conc_boundaries = {
    "vac" : dict(YSZ = - ( i_a / e * NA * charges["vac"] ) / vac["diffusivity"], anode = 0.0),
    "elec" : dict(YSZ = 0.0, anode = - (i_a / e * NA * charges["elec"]) / vac["diffusivity"]),
    "yzr" : dict(YSZ = 0.0, anode = 0.0),
}

print(conc_boundaries["elec"]["YSZ"])

0.0


In [76]:
nx = 100

dx = 1.0

x = np.arange(0, nx, dx)

In [81]:
#initialize the concentrations of each species and electrostatic potential of the system

# ------------------------------------------------------------------------------
# Vacancy concentrations [mol/m³]
# ------------------------------------------------------------------------------

c_vac = np.zeros(nx)

c_vac[0:int(nx/2)] = vac["bulk_YSZ_conc"]
c_vac[int(nx/2):nx] = vac["bulk_anode_conc"]

c_vac = c_vac / c_ref  # normalize by reference concentration

# ------------------------------------------------------------------------------
# Yttrium-stabilized Zirconium concentrations [mol/m³]
# ------------------------------------------------------------------------------

c_yzr = np.zeros(nx)

c_yzr[0:int(nx/2)] = yzr["bulk_YSZ_conc"]
c_yzr[int(nx/2):nx] = yzr["bulk_anode_conc"]

c_yzr = c_yzr / c_ref  # normalize by reference concentration

# ------------------------------------------------------------------------------
# Electron concentrations [mol/m³]
# ------------------------------------------------------------------------------

c_elec = np.zeros(nx)

c_elec[0:int(nx/2)] = elec["bulk_YSZ_conc"]
c_elec[int(nx/2):nx] = elec["bulk_anode_conc"]

c_elec = c_elec / c_ref  # normalize by reference concentration

# ------------------------------------------------------------------------------
# Initialize the electrostatic potential field
# ------------------------------------------------------------------------------

varphi = np.ones(nx) #potential field

In [85]:
rho = e * NA * ( charges["vac"] * c_vac + charges["elec"] * c_elec + charges["yzr"] * c_yzr )
print(rho/rho_ref)

rho /= rho_ref  # normalize by reference charge density
print("rho", rho)

for i in rho:
    rho[i] = i * 0.01

print(rho)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
rho [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]


TypeError: 'int' object is not iterable

In [83]:
# function to calculate the potential, varphi 
def gauss_seidel_1D(varphi, rho, maxiter = 100000, tol = 1e-4):
    
    # Get the length of potential array (1-D)
    nx = len(varphi)
    
    # Iterate until convergence or maximum iterations reached
    for it in range(maxiter):

        varphi_old = varphi.copy()

        varphi[0] = 0.0
        
        varphi[-1] = varphi[-2] + dx * i_a/sigma

        # Update the potential using the Gauss-Seidel formula
        for j in range(1, nx-1):
                varphi[j] = 0.5 * (varphi[j-1] + varphi[j+1] - rho[j] * dx**2 / epsilon)
        
        # Check for convergence
        if np.max(np.abs(varphi - varphi_old)) < tol:
            print(f"Converged after {it} iterations.")
            return varphi
    
    print("Maximum iterations reached without convergence.")
    return varphi

test = gauss_seidel_1D(varphi, rho)

print(test)

Converged after 4578 iterations.
[0.         0.00637937 0.01275553 0.01912687 0.02549178 0.03184867
 0.03819594 0.04453198 0.05085521 0.05716403 0.06345687 0.06973213
 0.07598824 0.08222364 0.08843675 0.09462602 0.1007899  0.10692683
 0.11303528 0.11911371 0.12516062 0.13117447 0.13715376 0.143097
 0.1490027  0.15486938 0.16069558 0.16647983 0.17222069 0.17791673
 0.18356652 0.18916866 0.19472174 0.20022438 0.2056752  0.21107286
 0.216416   0.22170328 0.22693341 0.23210507 0.23721697 0.24226785
 0.24725645 0.25218153 0.25704187 0.26183625 0.2665635  0.27122244
 0.27581191 0.28033077 0.28477792 0.28915224 0.29345265 0.2976781
 0.30182754 0.30589994 0.3098943  0.31380964 0.31764499 0.3213994
 0.32507195 0.32866175 0.3321679  0.33558955 0.33892586 0.34217601
 0.34533921 0.34841468 0.35140167 0.35429945 0.35710733 0.35982461
 0.36245063 0.36498476 0.36742638 0.36977491 0.37202978 0.37419044
 0.37625637 0.37822709 0.38010211 0.38188099 0.38356331 0.38514867
 0.3866367  0.38802705 0.38931939

In [70]:
from scipy.linalg import solve

def gauss(A, b, x, n):
    L = np.tril(A)
    U = A - L
    for i in range(n):
        x = np.linalg.solve(L, b - np.dot(U, x))
    return x

A = np.zeros((nx, nx))
b = np.zeros(nx)

#Dirichlet boundary conditions for the electrolyte
A[0, 0] = 1.0
b[0] = 0.0

#Setting up the matrix A and vector b
for i in range(1, nx-1):
    A[i, i-1] = -1.0
    A[i, i] = 2.0
    A[i, i+1] = -1.0
    b[i] = rho[i] * dx**2 / epsilon

#Neumann boundary conditions for the anode
A[-1, -2] = -1.0
A[-1, -1] = 1.0
b[-1] = dx * i_a / sigma

print(A)
print(b)

print (gauss(A, b, x, nx))
print(solve(A, b))

# function to calculate the potential, varphi 
def gauss_seidel_1D(varphi, rho, maxiter = 100000, tol = 1e-4):
    
    # Get the length of potential array (1-D)
    nx = len(varphi)
    
    # Iterate until convergence or maximum iterations reached
    for it in range(maxiter):

        varphi_old = varphi.copy()

        varphi[0] = 0.0
        
        varphi[-1] = varphi[-2] + dx * i_a/sigma

        # Update the potential using the Gauss-Seidel formula
        for j in range(1, nx-1):
                varphi[j] = 0.5 * (varphi[j-1] + varphi[j+1] - rho[j] * dx**2 / epsilon)
        
        # Check for convergence
        if np.max(np.abs(varphi - varphi_old)) < tol:
            print(f"Converged after {it} iterations.")
            return varphi
    
    print("Maximum iterations reached without convergence.")
    return varphi

test = gauss_seidel_1D(varphi, rho)

print(test)

[[ 1.  0.  0. ...  0.  0.  0.]
 [-1.  2. -1. ...  0.  0.  0.]
 [ 0. -1.  2. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ...  2. -1.  0.]
 [ 0.  0.  0. ... -1.  2. -1.]
 [ 0.  0.  0. ...  0. -1.  1.]]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
[ 0.          1.          2.          3.          4.          5.
  6.          7.          8.          9.         10.         11.
 12.         13.         14.         15.         16.         17.
 18.         19.         20.         21.         22.         23.
 24.         25.         26.         27.         28.         29.
 30.         31.         32.         32.99999999 33.99999998 34.99999996
 35.99999992 36.99999985 37.99999973 38.99999949 39.99999908 40.99999835
 41.99999711 42.99999501 43.999

In [71]:
nsteps = 10

A = np.zeros((nx, nx))
b = np.zeros(nx)

#Dirichlet boundary conditions for the electrolyte
A[0, 0] = 1.0
b[0] = 0.0

#Setting up the matrix A and vector b
for i in range(1, nx-1):
    A[i, i-1] = -1.0
    A[i, i] = 2.0
    A[i, i+1] = -1.0
    b[i] = rho[i] * dx**2 / epsilon

#Neumann boundary conditions for the anode
A[-1, -2] = -1.0
A[-1, -1] = 1.0
b[-1] = dx * i_a / sigma

print(A)
print(b)

def gauss(A, b, x, n):
    for t in range(nsteps):
        for _ in range(n):
            for i in range(len(b)):
                sum_ = sum(A[i, j] * x[j] for j in range(len(b)) if j != i)
                x[i] = (b[i] - sum_) / A[i, i]
    return x

print(gauss(A, b, varphi, nx))
print(solve(A, b))

[[ 1.  0.  0. ...  0.  0.  0.]
 [-1.  2. -1. ...  0.  0.  0.]
 [ 0. -1.  2. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ...  2. -1.  0.]
 [ 0.  0.  0. ... -1.  2. -1.]
 [ 0.  0.  0. ...  0. -1.  1.]]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

In [72]:
import numpy as np
from scipy.linalg import solve

In [73]:
nx = 5

domain = 5.0

x = np.linspace(0, domain, nx)

rho = np.copy(x) # Initial charge density

varphi = np.zeros(nx)  # Initial potential array

epsilon = 2 #8.854e-12 * 40.0 # Permittivity of free space (F/m) 

dx = 1

i_a = 0  # Current density (A/m)

sigma = 1.0 # Conductivity (S/m)

In [74]:
def gauss(A, b, x, n):
    L = np.tril(A)
    U = A - L
    for i in range(n):
        x = np.linalg.solve(L, b - np.dot(U, x))
    return x

In [75]:
A = np.zeros((nx, nx))
b = np.zeros(nx)

#Dirichlet boundary conditions for the electrolyte
A[0, 0] = 1.0
b[0] = 0.0

#Setting up the matrix A and vector b
for i in range(1, nx-1):
    A[i, i-1] = -1.0
    A[i, i] = 2.0
    A[i, i+1] = -1.0
    b[i] = rho[i] * dx**2 / epsilon

#Neumann boundary conditions for the anode
A[-1, -2] = -1.0
A[-1, -1] = 1.0
b[-1] = dx * i_a / sigma

print(A)
print(b)

print (gauss(A, b, x, nx))
print(solve(A, b))

[[ 1.  0.  0.  0.  0.]
 [-1.  2. -1.  0.  0.]
 [ 0. -1.  2. -1.  0.]
 [ 0.  0. -1.  2. -1.]
 [ 0.  0.  0. -1.  1.]]
[0.    0.625 1.25  1.875 0.   ]
[0.         2.77832031 5.21728516 6.74926758 6.74926758]
[0.    3.75  6.875 8.75  8.75 ]
